produce

In [1]:
import os
import pandas as pd
import numpy as np
import pickle
from tqdm import tqdm
from PyBioMed import Pyprotein
from PyBioMed.PyProtein.GetProteinFromUniprot import GetProteinSequence
from PyBioMed.PyGetMol import Getmol
from PyBioMed.PyGetMol.Getmol import ReadMolFromSDF
from PyBioMed import Pymolecule
from rdkit import Chem
#from PyBioMed.PyMolecule import topology
from PyBioMed.PyMolecule import connectivity
from PyBioMed.PyMolecule.cats2d import CATS2D


Define drug_descriptor and target_descriptor

In [2]:
def drug_descriptor(drug_smi, drug_feature):
    try:
        temp = {}
        mol = Pymolecule.PyMolecule()
        mol.ReadMolFromSmile(drug_smi)
        Mol = Chem.MolFromSmiles(drug_smi)
        temp.update(connectivity.GetConnectivity(Mol))
        temp.update(CATS2D(Mol,PathLength = 10,scale = 3))
#        temp.update(topology.GetTopology(Mol))
        temp.update(mol.GetEstate())
        temp.update(mol.GetCharge())
        temp.update(mol.GetKappa())
        temp = pd.DataFrame.from_dict(temp, orient='index').T
        temp = temp.iloc[:][drug_feature].values
        return temp
    except:
        return 'nan'
    
    
def target_descriptor(protein, target_feature):
    try:
        temp = {}
        protein_class = Pyprotein.PyProtein(protein)
        temp.update(protein_class.GetAAComp())
        temp.update(protein_class.GetDPComp())
        temp.update(protein_class.GetCTD())
        temp.update(protein_class.GetMoreauBrotoAuto())
        temp.update(protein_class.GetQSO())
        temp.update(protein_class.GetSOCN())
        temp = pd.DataFrame.from_dict(temp, orient='index').T
        temp = temp.iloc[:][target_feature].values
        return temp
    except:
        return 'nan'

In [3]:
target_database = pd.read_csv('C:\\Users\\lab619\\Music\\DeepDTI\\code\\drug_target_descriptor\\biomarker_CT.txt', sep='\t')
drug_feature    = pd.read_csv('C:\\Users\\lab619\\Music\\DeepDTI\\code\\drug_target_descriptor\\drug feature name_1.txt', sep='\t' )
target_feature  = pd.read_csv('C:\\Users\\lab619\\Music\\DeepDTI\\code\\drug_target_descriptor\\target feature name.txt', sep='\t' )
L1000           = pd.read_csv('C:\\Users\\lab619\\Music\\DeepDTI\\code\\drug_target_descriptor\\L1000_Sense.txt', sep='\t' )

In [4]:
cell_line = ['A549']                               # input the cell line U want
e = pd.DataFrame(columns=L1000.columns)
for i in cell_line:
    e = e.append(L1000[L1000['cell_id']==i])
e = e.drop_duplicates(subset=['pert_iname'])
e.head()


,pubchem_cid,cell_id,pert_iname,canonical_smiles,PSME1,ATF1,RHEB,FOXO3,RHOA,IL1B,...,CD320,MLLT11,CEBPZ,CBR3,DB_id,LD50(mol/kg),FDA,pert_dose,pert_id,Sensitivity
37,9931954,A549,rucaparib,CNCC1=CC=C(C=C1)C2=C3CCNC(=O)C4=CC(=CC(=C34)N2)F,0.1265,0.2867,-0.6933,-0.1626,0.3490,-1.5586,...,0.4957,0.4809,-1.2558,-0.4108,NaN,NaN,Approved,10.0,BRD-K88560311,0.271891
38,702558,A549,RG-108,C1=CC=C2C(=C1)C(=O)N(C2=O)C(CC3=CNC4=CC=CC=C43...,-0.6135,-0.6865,-0.7815,-0.4501,-0.9175,1.4609,...,-0.6290,0.3640,-0.8461,-0.4485,NaN,NaN,NaN,80.0,BRD-K89391146,-0.842937
39,176158,A549,SB-216763,CN1C=C(C2=CC=CC=C21)C3=C(C(=O)NC3=O)C4=C(C=C(C...,-0.6150,-0.7083,-2.5591,-0.4008,-1.8657,0.2811,...,1.1554,1.2791,0.1105,-0.7735,NaN,NaN,NaN,10.0,BRD-K59184148,0.210876
40,2726824,A549,XAV-939,C1CSCC2=C1NC(=NC2=O)C3=CC=C(C=C3)C(F)(F)F,-0.8123,-0.2720,-0.1691,-0.2443,0.9580,-0.1087,...,-0.0733,1.0046,0.0937,0.6786,NaN,NaN,NaN,11.1,BRD-K12762134,-0.674420
41,5282054,A549,cerulenin,CC=CCC=CCCC(=O)C1C(O1)C(=O)N,-0.8754,-0.3467,0.7323,0.0109,-0.3706,-0.1531,...,0.1600,0.3911,0.1232,1.3101,NaN,NaN,NaN,44.4,BRD-K52075040,0.657295


In [5]:
drug_lst = [  drug_descriptor(  e.iloc[i]['canonical_smiles']  , drug_feature.iloc[:,0].tolist()  )  for i in tqdm(range(len(e)))  ]   # convert to feature for eaach drug
e.insert(0,'descriptor', drug_lst)
e.drop(index=[  e.index[i]  for i in  range(len(e)) if    np.isnan(e.iloc[i,0]).any()     ]  , inplace=True)                           # descriptor 有 nan 的 drug 去掉 


with open('C:\\Users\\lab619\\Music\\DeepDTI\\code\\drug_target_descriptor\\FDA_drug_CT.pickle', 'wb') as file:
    pickle.dump(e, file, protocol=2)

print("Drug numeber : %d  feature numeber : %d    "  %( len(e) , e.iloc[0,0].shape[1]     )             )

 33%|███▎      | 285/873 [00:51<02:26,  4.02it/s]C:\Users\lab619\anaconda3\envs\myenv\lib\site-packages\PyBioMed\PyMolecule\charge.py:454: RuntimeWarning: invalid value encountered in greater
  return round(numpy.mean(cc[cc>0]),3)
C:\Users\lab619\anaconda3\envs\myenv\lib\site-packages\numpy\core\fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\Users\lab619\anaconda3\envs\myenv\lib\site-packages\numpy\core\_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\Users\lab619\anaconda3\envs\myenv\lib\site-packages\PyBioMed\PyMolecule\charge.py:426: RuntimeWarning: invalid value encountered in greater
  return round(sum(cc[cc>0]),3)
C:\Users\lab619\anaconda3\envs\myenv\lib\site-packages\PyBioMed\PyMolecule\charge.py:483: RuntimeWarning: invalid value encountered in less
  return round(sum(cc[cc<0]),3)
C:\Users\lab619\anaconda3\envs\myenv\lib\site-packages\PyBioMed\PyMolecule\charge.py:632: RuntimeW

Drug numeber : 872  feature numeber : 363    


In [6]:
target_lst = [    target_descriptor(GetProteinSequence(target_database.iloc[i]['protein']), target_feature.iloc[:,0].tolist())  for i in  tqdm(range(len(target_database))) ]   # convert to feature for eaach target
target_database['descriptor'] = target_lst

with open('C:\\Users\\lab619\\Music\\DeepDTI\\code\\drug_target_descriptor\\target_gene_CT.pickle', 'wb') as file:
    pickle.dump(target_database, file, protocol=2)
print("Target numeber : %d  feature numeber : %d    "  %( len(target_database) , target_database.iloc[0,2].shape[1]  )  )

100%|██████████| 4/4 [00:08<00:00,  2.22s/it]

Target numeber : 4  feature numeber : 996    
